In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
ls data/sports_scores/

astros_1A10.csv*   Rice_10H80.csv*    Tex_15E40.csv*
dynamo_10H30.csv*  rockets_1A10.csv*  UH_10H50.csv*


In [3]:
ls data/crime_beat/

crime_10H30_10_17.csv*  crime_10H80_10_17.csv*  crime_1A10_10_17.csv*
crime_10H50_10_17.csv*  crime_15E40_10_17.csv*


In [4]:
games = 'data/sports_scores/astros_1A10.csv'
crime = 'data/crime_beat/crime_1A10_10_17.csv'

# scores df

In [5]:
df = pd.read_csv(games,index_col='date').sort_index(ascending=True)

## win column
df['win'] =  np.where(df['team_score'] > df['opp_score'],1,0)

# only oposing name and win boolean
df = df[['opposing','win']]

# show

df.head()

,opposing,win
date,,
2010-04-05,SFG,0
2010-04-06,SFG,0
2010-04-07,SFG,0
2010-04-09,PHI,0
2010-04-10,PHI,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 658 entries, 2010-04-05 to 2017-10-29
Data columns (total 2 columns):
opposing    658 non-null object
win         658 non-null int64
dtypes: int64(1), object(1)
memory usage: 15.4+ KB


# Crime df

In [7]:
crime_df = pd.read_csv(crime).sort_index(ascending=True)

crime_df['Date_Hour'] = pd.to_datetime(crime_df['Date_Hour'])
crime_df['Date_Hour'] = crime_df['Date_Hour'].dt.date

crime_df = crime_df.set_index('Date_Hour')


# selectd columns
crime_df = crime_df[['OffenseType']]

# count Offense type given date
date_crimes = crime_df.groupby(crime_df.index)['OffenseType',].count()

## show
date_crimes.head()

,OffenseType
Date_Hour,
2010-01-01,11
2010-01-02,11
2010-01-03,8
2010-01-04,6
2010-01-05,5


In [8]:
#crime_df = pd.get_dummies(crime_df,drop_first=True)

In [9]:
#crime_df.info()

# Days Df
- create a df with everyday timestamp from Jan 2010 to Dec 2017

In [10]:
days = pd.date_range(start='01/01/2010', end='12/31/2017')
days = pd.DataFrame(days)

days.columns = ['days']
days = days.set_index('days').sort_index(ascending=True)
days.head()

""
days
2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05


In [11]:
365 * 8

2920

In [12]:
len(days)/365

8.005479452054795

In [13]:
days.tail()

""
days
2017-12-27
2017-12-28
2017-12-29
2017-12-30
2017-12-31


# Merge days df with crimes_df

In [14]:
calendar_crimes = pd.merge(days,date_crimes,  left_index=True, right_index=True, how='left')
calendar_crimes.head()

,OffenseType
days,
2010-01-01,11.0
2010-01-02,11.0
2010-01-03,8.0
2010-01-04,6.0
2010-01-05,5.0


In [15]:
calendar_crimes.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 1 columns):
OffenseType    2891 non-null float64
dtypes: float64(1)
memory usage: 125.7 KB


In [16]:
calendar_crimes.OffenseType.value_counts(dropna=False)

 5.0     459
 4.0     453
 3.0     402
 6.0     353
 2.0     282
 7.0     274
 8.0     205
 1.0     149
 9.0     118
 10.0     61
 11.0     35
NaN       31
 12.0     30
 13.0     21
 14.0     19
 15.0      7
 18.0      5
 16.0      4
 17.0      4
 21.0      3
 20.0      3
 19.0      2
 22.0      1
 36.0      1
Name: OffenseType, dtype: int64

## Change nans with 0

In [17]:
calendar_crimes.OffenseType.replace( np.nan,0, inplace=True)

In [18]:
calendar_crimes.OffenseType.value_counts(dropna=False)

5.0     459
4.0     453
3.0     402
6.0     353
2.0     282
7.0     274
8.0     205
1.0     149
9.0     118
10.0     61
11.0     35
0.0      31
12.0     30
13.0     21
14.0     19
15.0      7
18.0      5
16.0      4
17.0      4
21.0      3
20.0      3
19.0      2
22.0      1
36.0      1
Name: OffenseType, dtype: int64

# merge calendar_crimes with score

In [19]:
merge_data = pd.merge(calendar_crimes,df,  left_index=True, right_index=True, how='left')

# change column names
#merge_data.columns = ['offenses','away_team','win']

merge_data.head()

,OffenseType,opposing,win
2010-01-01,11.0,NaN,NaN
2010-01-02,11.0,NaN,NaN
2010-01-03,8.0,NaN,NaN
2010-01-04,6.0,NaN,NaN
2010-01-05,5.0,NaN,NaN


In [20]:
merge_data['opposing'].value_counts(dropna=False)

NaN            2265
TEX              57
SEA              49
OAK              47
LAA              47
CIN              30
STL              28
MIL              26
PIT              25
TBR              24
CHC              24
KCR              19
CLE              19
ARI              19
BOS              18
TOR              18
DET              17
BAL              17
CHW              16
NYY              16
WSN              16
ATL              15
MIN              15
SFG              14
NYM              13
COL              13
LAD              13
SDP              11
PHI              10
MIA               6
FLA               6
New York          4
Los Angeles       3
Kansas City       2
Boston            1
Name: opposing, dtype: int64

## Cleanup colums

In [21]:
merge_data.columns  = ['crime', 'away_team', 'won']

In [22]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2923 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2923 non-null float64
away_team    658 non-null object
won          658 non-null float64
dtypes: float64(2), object(1)
memory usage: 91.3+ KB


In [23]:
merge_data.crime.value_counts(dropna=False)

5.0     459
4.0     453
3.0     402
6.0     353
2.0     282
7.0     275
8.0     205
1.0     149
9.0     118
10.0     61
11.0     35
0.0      31
12.0     30
13.0     21
14.0     19
15.0      7
18.0      5
16.0      4
17.0      4
21.0      3
20.0      3
19.0      2
22.0      1
36.0      1
Name: crime, dtype: int64

In [24]:
merge_data.crime = merge_data.crime.astype(int)

In [25]:
merge_data.crime.value_counts(dropna=False)

5     459
4     453
3     402
6     353
2     282
7     275
8     205
1     149
9     118
10     61
11     35
0      31
12     30
13     21
14     19
15      7
18      5
17      4
16      4
21      3
20      3
19      2
36      1
22      1
Name: crime, dtype: int64

In [26]:
merge_data

,crime,away_team,won
2010-01-01,11,NaN,NaN
2010-01-02,11,NaN,NaN
2010-01-03,8,NaN,NaN
2010-01-04,6,NaN,NaN
2010-01-05,5,NaN,NaN
2010-01-06,7,NaN,NaN
2010-01-07,8,NaN,NaN
2010-01-08,4,NaN,NaN
2010-01-09,10,NaN,NaN
2010-01-10,8,NaN,NaN


# Display non-nans

In [27]:
merge_data.dropna(subset=['away_team', 'won'])

,crime,away_team,won
2010-04-05,12,SFG,0.0
2010-04-06,7,SFG,0.0
2010-04-07,6,SFG,0.0
2010-04-09,9,PHI,0.0
2010-04-10,9,PHI,0.0
2010-04-11,6,PHI,0.0
2010-04-20,2,FLA,1.0
2010-04-21,1,FLA,1.0
2010-04-22,7,FLA,0.0
2010-04-23,8,PIT,1.0


In [28]:
merge_data.to_csv('data/final/Astros1017_final.csv')

In [29]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2923 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2923 non-null int64
away_team    658 non-null object
won          658 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 91.3+ KB
